<h1>BLOCKCHAIN</h1>


<br>
<p><b>Blockchain</b> adalah catatan transaksi digital berdasarkan strukturnya, di mana catatan individu, yang disebut blok, dihubungkan bersama dalam satu daftar, yang disebut chain (rantai).</p>
<br>
<p>Seluruh pengembangan proyek blockchain terdiri dari tiga komponen utama, yaitu</p>

<ul>
    <li><b>Client</b> -> Client/Klien adalah orang yang akan membeli barang dari vendor lain. Klien sendiri dapat menjadi vendor dan akan menerima uang dari orang lain terhadap barang yang dia suplai. Kami berasumsi di sini bahwa klien dapat menjadi pemasok dan penerima TPCoin. Jadi, kami akan membuat kelas klien dalam kode kami yang memiliki kemampuan untuk mengirim dan menerima uang.</li><br>
    <li><b>Miner</b> -> Miner/Penambang adalah orang yang mengambil transaksi dari kumpulan transaksi dan mengumpulkannya dalam satu blok. Penambang harus memberikan bukti kerja yang valid untuk mendapatkan hadiah penambangan. Semua uang yang dikumpulkan penambang sebagai biaya akan dia simpan. Dia dapat membelanjakan uang itu untuk membeli barang atau jasa dari vendor terdaftar lainnya di jaringan, seperti yang dilakukan oleh Klien yang dijelaskan di atas.</li>
    
</ul>
<br>
<br>

In [14]:
from IPython.display import Image
from IPython.core.display import HTML 
Image(url="https://miro.medium.com/max/1400/0*EBXqB5wJTWFeaT04.gif")

<h2>1. Developing Client</h2>
<p>Klien adalah seseorang yang memegang TPCoin dan mentransaksikannya untuk barang/jasa dari vendor lain di jaringan termasuk miliknya. Kita harus mendefinisikan kelas Klien untuk tujuan ini. Untuk membuat identifikasi unik global untuk klien, kami menggunakan PKI (Public Key Infrastructure). Dalam bab ini, mari kita bicarakan ini secara rinci.



Klien harus dapat mengirim uang dari dompetnya ke orang lain yang dikenal. Demikian pula, klien harus dapat menerima uang dari pihak ketiga. Untuk membelanjakan uang, klien akan membuat transaksi yang menentukan nama pengirim dan jumlah yang harus dibayar. Untuk menerima uang, klien akan memberikan identitasnya kepada pihak ketiga pada dasarnya adalah pengirim uang. Kami tidak menyimpan jumlah saldo uang yang dimiliki klien di dompetnya. Selama transaksi, kami akan menghitung saldo aktual untuk memastikan bahwa klien memiliki saldo yang cukup untuk melakukan pembayaran.



Untuk mengembangkan kelas Klien dan untuk sisa kode dalam proyek, kita perlu mengimpor banyak liblary Python. Ini tercantum di bawah ini</p>

In [7]:
import hashlib
import random
import string
import json
import binascii
import numpy as np
import pandas as pd
import pylab as pl
import logging
import datetime
import collections

# following imports are required by PKI
import Crypto
import Crypto.Random
from Crypto.Hash import SHA
from Crypto.PublicKey import RSA
from Crypto.Signature import PKCS1_v1_5


<br>
<br>
<h2>2. Client Class</h2>
<p>class Client menghasilkan kunci pribadi dan publik dengan menggunakan algoritma RSA. Pembaca yang tertarik dapat merujuk ke tutorial ini untuk implementasi RSA. Selama inisialisasi objek, kita membuat kunci privat dan publik dan menyimpan nilainya dalam variabel instan.</p>

```
self._private_key = RSA.generate(1024, random)
self._public_key = self._private_key.publickey()
```
<br><p><b>NOTE</b> : Perhatikan bahwa Anda tidak boleh kehilangan kunci pribadi Anda. Untuk penyimpanan catatan, kunci pribadi yang dihasilkan dapat disalin pada penyimpanan eksternal yang aman atau Anda cukup menuliskan representasi ASCII.

Kunci publik yang dihasilkan akan digunakan sebagai identitas klien. Untuk ini, kami mendefinisikan properti yang disebut identitas yang mengembalikan representasi HEX dari kunci publik.</p>
```
@property
   def identity(self):
      return
binascii.hexlify(self._public_key.exportKey(format='DER')).decode('ascii')
```
<br>
<p>Identity unik untuk setiap Client dan dapat dibuat tersedia untuk umum. Siapa pun akan dapat mengirim mata uang virtual kepada Anda menggunakan identitas ini dan itu akan ditambahkan ke dompet Anda.</p>

In [ ]:
class Client:
   def __init__(self):
      random = Crypto.Random.new().read
      self._private_key = RSA.generate(1024, random)
      self._public_key = self._private_key.publickey()
      self._signer = PKCS1_v1_5.new(self._private_key)

   @property
   def identity(self):
      return
binascii.hexlify(self._public_key.exportKey(format='DER')).decode('ascii')

<h3>Testing client</h3>

In [ ]:
Dinesh = Client()
print(Dinesh.identity)

<p>di atas membuat instance dari Klien dan menetapkannya ke variabel Dinesh. Kami mencetak kunci publik Dinesh dengan memanggil metode identitasnya.</p>

```
30819f300d06092a864886f70d010101050003818d0030818902818100b547fafceeb131e07
0166a6b23fec473cce22c3f55c35ce535b31d4c74754fecd820aa94c1166643a49ea5f49f72
3181ff943eb3fdc5b2cb2db12d21c06c880ccf493e14dd3e93f3a9e175325790004954c34d3
c7bc2ccc9f0eb5332014937f9e49bca9b7856d351a553d9812367dc8f2ac734992a4e6a6ff6
6f347bd411d07f0203010001
```

<br>
<br>
<h2>3. Transaction Class</h2>
<p>mari kita buat kelas Transaksi sehingga klien dapat mengirim uang kepada seseorang. Perhatikan bahwa klien dapat menjadi pengirim atau penerima uang. Ketika Anda ingin menerima uang, beberapa pengirim lain akan membuat transaksi dan menentukan alamat publik Anda di dalamnya. Kami mendefinisikan inisialisasi kelas transaksi sebagai berikut:</p>

```
def __init__(self, sender, recipient, value):
   self.sender = sender
   self.recipient = recipient
   self.value = value
   self.time = datetime.datetime.now()
```
<p>init mengambil tiga parameter - kunci publik pengirim, kunci publik penerima, dan jumlah yang akan dikirim. Ini disimpan dalam variabel instan untuk digunakan oleh metode lain. Selain itu, kami membuat satu variabel lagi untuk menyimpan waktu transaksi.



Selanjutnya, kami menulis metode utilitas yang disebut to_dict yang menggabungkan keempat variabel instan yang disebutkan di atas dalam objek kamus. Ini hanya untuk menempatkan seluruh informasi transaksi yang dapat diakses melalui satu variabel.



Seperti yang Anda ketahui dari tutorial sebelumnya bahwa blok pertama di blockchain adalah blok Genesis. Blok Genesis berisi transaksi pertama yang diprakarsai oleh pencipta blockchain. Identitas orang ini mungkin dirahasiakan seperti dalam kasus Bitcoin. Jadi pada saat transaksi pertama ini dibuat, pencipta boleh saja mengirimkan identitasnya sebagai Genesis. Jadi, saat membuat kamus, kami memeriksa apakah pengirimnya adalah Genesis dan jika demikian, kami hanya menetapkan beberapa nilai string ke variabel identitas; jika tidak, kami menetapkan identitas pengirim ke variabel identitas.</p>

```
if self.sender == "Genesis":
   identity = "Genesis"
else:
   identity = self.sender.identity
```

<p>dengan construct dictionary</p>

```
return collections.OrderedDict(
       {
         'sender'    : identity,
         'recipient' : self.recipient,
         'value'     : self.value,
         'time'      : self.time
       }
)
```

In [ ]:
def to_dict(self):
   if self.sender == "Genesis":
      identity = "Genesis"
   else:
      identity = self.sender.identity

   return collections.OrderedDict(
          {
            'sender'    : identity,
            'recipient' : self.recipient,
            'value'     : self.value,
            'time'      : self.time
          }
   )

<p>Terakhir, kita akan menandatangani objek kamus ini menggunakan kunci pribadi pengirim. Seperti sebelumnya, kami menggunakan PKI bawaan dengan algoritma SHA. Tanda tangan yang dihasilkan didekodekan untuk mendapatkan representasi ASCII untuk dicetak dan disimpan di blockchain kami. Kode metode sign_transaction ditampilkan di sini</p>

In [ ]:
def sign_transaction(self):
   private_key = self.sender._private_key
   signer = PKCS1_v1_5.new(private_key)
   h = SHA.new(str(self.to_dict()).encode('utf8'))
   return binascii.hexlify(signer.sign(h)).decode('ascii')

<h3>Testing Transaction Class</h3>
<p>Untuk tujuan ini, kami akan membuat dua pengguna, yang disebut Dinesh dan Ramesh. Dinesh akan mengirimkan 5 TPCoin ke Ramesh. Untuk ini pertama kami membuat klien yang disebut Dinesh dan Ramesh.</p>

In [ ]:
Dinesh = Client()
Ramesh = Client()

<p>Ingatlah bahwa ketika Anda membuat instance kelas Klien, kunci publik dan pribadi yang unik untuk klien akan dibuat. Saat Dinesh mengirim pembayaran ke Ramesh, dia akan membutuhkan kunci publik Ramesh yang diperoleh dengan menggunakan properti identitas klien.



Jadi, kami akan membuat instance transaksi menggunakan kode berikut:</p>

In [ ]:
t = Transaction(
   Dinesh,
   Ramesh.identity,
   5.0
)

<p><b>NOTE</b> : bahwa parameter pertama adalah pengirim, parameter kedua adalah kunci publik penerima dan parameter ketiga adalah jumlah yang akan ditransfer. Metode sign_transaction mengambil kunci pribadi pengirim dari parameter pertama untuk menyanyikan transaksi.



Setelah objek transaksi dibuat, Anda akan menandatanganinya dengan memanggil metode sign_transaction-nya. Metode ini mengembalikan tanda tangan yang dihasilkan dalam format yang dapat dicetak. Kami membuat dan mencetak tanda tangan menggunakan dua baris kode berikut:</p>

In [ ]:
signature = t.sign_transaction()
print (signature) 

<p>output : </p>

```
7c7e3c97629b218e9ec6e86b01f9abd8e361fd69e7d373c38420790b655b9abe3b575e343c7
13703ca1aee781acd7157a0624db3d57d7c2f1172730ee3f45af943338157f899965856f6b0
0e34db240b62673ad5a08c8e490f880b568efbc36035cae2e748f1d802d5e8e66298be826f5
c6363dc511222fb2416036ac04eb972
```


<br>
<br>
<h2>4. Multiple Transactions</h2>
<p>Transaksi yang dilakukan oleh berbagai klien diantrekan dalam sistem; penambang mengambil transaksi dari antrian ini dan menambahkannya ke blok. Mereka kemudian akan menambang blok dan penambang yang menang akan memiliki hak istimewa untuk menambahkan blok ke blockchain dan dengan demikian mendapatkan sejumlah uang untuk dirinya sendiri.



Kami akan menjelaskan proses penambangan ini nanti ketika kami membahas pembuatan blockchain. Sebelum kita menulis kode untuk beberapa transaksi, mari kita tambahkan fungsi utilitas kecil untuk mencetak isi dari transaksi yang diberikan.</p>

<h3>Displaying Transaction</h3>
<p>Fungsi display_transaction menerima satu parameter tipe transaksi. Objek kamus dalam transaksi yang diterima disalin ke variabel sementara yang disebut dict dan menggunakan kunci kamus, berbagai nilai dicetak di konsol.</p>

In [ ]:
def display_transaction(transaction):
   #for transaction in transactions:
   dict = transaction.to_dict()
   print ("sender: " + dict['sender'])
   print ('-----')
   print ("recipient: " + dict['recipient'])
   print ('-----')
   print ("value: " + str(dict['value']))
   print ('-----')
   print ("time: " + str(dict['time']))
   print ('-----')

<h3>Transaction Queue</h3>
<p>Untuk membuat antrian, kami mendeklarasikan variabel daftar global yang disebut transaksi sebagai berikut:</p>

In [ ]:
transactions = []


<h3>Creating Multiple Clients</h3>
<p>Now, we will start creating transactions. First, we will create four clients who will send money to each other for obtaining various services or goods from others.</p>

In [ ]:
Dinesh = Client()
Ramesh = Client()
Seema = Client()
Vijay = Client()

<p>Pada titik ini memiliki empat klien bernama Dinesh, Ramesh, Seema, dan Vijay. Saat ini kami berasumsi bahwa masing-masing klien ini menyimpan beberapa TPCoin di dompet mereka untuk bertransaksi. Identitas masing-masing klien ini akan ditentukan dengan menggunakan properti identitas dari objek-objek ini.</p>

<h3>Creating First Transaction</h3>

In [ ]:
t1 = Transaction(
   Dinesh,
   Ramesh.identity,
   15.0
)

Dalam transaksi ini Dinesh mengirimkan 5 TPCoin ke Ramesh. Agar transaksi berhasil, kita harus memastikan bahwa Dinesh memiliki cukup uang di dompetnya untuk pembayaran ini. Perhatikan bahwa, kita memerlukan transaksi genesis untuk memulai sirkulasi TPCoin di sistem. Anda akan menulis kode transaksi untuk transaksi genesis ini segera setelah Anda membaca.



Ini akan menandatangani transaksi menggunakan kunci pribadi Dinesh dan menambahkannya ke antrian transaksi sebagai berikut:

In [ ]:
t1.sign_transaction()
transactions.append(t1)

Setelah transaksi pertama dilakukan oleh Dinesh, kami akan membuat beberapa transaksi lagi antara klien yang berbeda yang kami buat di atas.


<h3>Adding More Transactions</h3>

sekarang kita akan membuat beberapa transaksi lagi, setiap transaksi memberikan beberapa TPCoin ke pihak lain. Ketika seseorang membelanjakan uang, dia tidak perlu memeriksa saldo yang cukup di dompet ini. Penambang bagaimanapun akan memvalidasi setiap transaksi untuk saldo yang dimiliki pengirim saat memulai transaksi.

Jika saldo tidak mencukupi, penambang akan menandai transaksi ini sebagai tidak valid dan tidak akan menambahkannya ke blok ini.

Kode berikut membuat dan menambahkan sembilan transaksi lagi ke antrian kami.

In [ ]:
t2 = Transaction(
   Dinesh,
   Seema.identity,
   6.0
)
t2.sign_transaction()
transactions.append(t2)
t3 = Transaction(
   Ramesh,
   Vijay.identity,
   2.0
)
t3.sign_transaction()
transactions.append(t3)
t4 = Transaction(
   Seema,
   Ramesh.identity,
   4.0
)
t4.sign_transaction()
transactions.append(t4)
t5 = Transaction(
   Vijay,
   Seema.identity,
   7.0
)
t5.sign_transaction()
transactions.append(t5)
t6 = Transaction(
   Ramesh,
   Seema.identity,
   3.0
)
t6.sign_transaction()
transactions.append(t6)
t7 = Transaction(
   Seema,
   Dinesh.identity,
   8.0
)
t7.sign_transaction()
transactions.append(t7)
t8 = Transaction(
   Seema,
   Ramesh.identity,
   1.0
)
t8.sign_transaction()
transactions.append(t8)
t9 = Transaction(
   Vijay,
   Dinesh.identity,
   5.0
)
t9.sign_transaction()
transactions.append(t9)
t10 = Transaction(
   Vijay,
   Ramesh.identity,
   3.0
)
t10.sign_transaction()
transactions.append(t10)

Saat Anda menjalankan kode di atas, Anda akan memiliki sepuluh transaksi dalam antrian bagi para penambang untuk membuat blok mereka.


<h3>Dumping Transactions</h3>

As a blockchain manager, you may periodically like to review the contents of transaction queue. For this purpose, you can use the display_transaction function that we developed earlier. To dump all transactions in the queue, just iterate the transactions list and for each referenced transaction, call the display_transaction function as shown here

In [ ]:
for transaction in transactions:
   display_transaction (transaction)
   print ('--------------')

Transaksi dipisahkan oleh garis putus-putus untuk membedakan. Jika Anda menjalankan kode di atas, Anda akan melihat daftar transaksi seperti yang ditunjukkan di bawah ini
```
sender:
30819f300d06092a864886f70d010101050003818d0030818902818100bb064c99c49214
4a9f463480273aba93ac1db1f0da3cb9f3c1f9d058cf499fd8e54d244da0a8dd6ddd329e
c86794b04d773eb4841c9f935ea4d9ccc2821c7a1082d23b6c928d59863407f52fa05d8b
47e5157f8fe56c2ce3279c657f9c6a80500073b0be8093f748aef667c03e64f04f84d311
c4d866c12d79d3fc3034563dfb0203010001
-----
recipient:
30819f300d06092a864886f70d010101050003818d0030818902818100be93b516b28c6e
674abe7abdb11ce0fdf5bb728b75216b73f37a6432e4b402b3ad8139b8c0ba541a72c8ad
d126b6e1a1308fb98b727beb63c6060356bb177bb7d54b54dbe87aee7353d0a6baa93977
04de625d1836d3f42c7ee5683f6703259592cc24b09699376807f28fe0e00ff882974484
d805f874260dfc2d1627473b910203010001
-----
value: 15.0
-----
time: 2019-01-14 16:18:01.859915
-----
--------------
sender:
30819f300d06092a864886f70d010101050003818d0030818902818100bb064c99c49214
4a9f463480273aba93ac1db1f0da3cb9f3c1f9d058cf499fd8e54d244da0a8dd6ddd329e
c86794b04d773eb4841c9f935ea4d9ccc2821c7a1082d23b6c928d59863407f52fa05d8b
47e5157f8fe56c2ce3279c657f9c6a80500073b0be8093f748aef667c03e64f04f84d311
c4d866c12d79d3fc3034563dfb0203010001
-----
recipient:
30819f300d06092a864886f70d010101050003818d0030818902818100a070c82b34ae14
3cbe59b3a2afde7186e9d5bc274955d8112d87a00256a35369acc4d0edfe65e8f9dc93fb
d9ee74b9e7ea12334da38c8c9900e6ced1c4ce93f86e06611e656521a1eab561892b7db0
961b4f212d1fd5b5e49ae09cf8c603a068f9b723aa8a651032ff6f24e5de00387e4d0623
75799742a359b8f22c5362e5650203010001
-----
value: 6.0
-----
time: 2019-01-14 16:18:01.860966
-----
--------------
sender:
30819f300d06092a864886f70d010101050003818d0030818902818100be93b516b28c6e
674abe7abdb11ce0fdf5bb728b75216b73f37a6432e4b402b3ad8139b8c0ba541a72c8ad
d126b6e1a1308fb98b727beb63c6060356bb177bb7d54b54dbe87aee7353d0a6baa93977
04de625d1836d3f42c7ee5683f6703259592cc24b09699376807f28fe0e00ff882974484
d805f874260dfc2d1627473b910203010001
-----
recipient:
30819f300d06092a864886f70d010101050003818d0030818902818100cba097c0854876
f41338c62598c658f545182cfa4acebce147aedf328181f9c4930f14498fd03c0af6b0cc
e25be99452a81df4fa30a53eddbb7bb7b203adf8764a0ccd9db6913a576d68d642d8fd47
452590137869c25d9ff83d68ebe6d616056a8425b85b52e69715b8b85ae807b84638d8f0
0e321b65e4c33acaf6469e18e30203010001
-----
value: 2.0
-----
time: 2019-01-14 16:18:01.861958
-----
--------------
```

Untuk singkatnya, saya hanya mencetak beberapa transaksi pertama dalam daftar. Pada kode di atas, kami mencetak semua transaksi yang dimulai dengan transaksi pertama kecuali transaksi genesis yang tidak pernah ditambahkan ke daftar ini. Karena transaksi ditambahkan ke blok secara berkala, Anda biasanya akan tertarik untuk melihat hanya daftar transaksi yang belum ditambang. Dalam hal ini, Anda perlu membuat loop for yang sesuai untuk beralih melalui transaksi yang belum ditambang

<br>
<br>

<h2>5. Block Class</h2>

Sebuah blok terdiri dari berbagai jumlah transaksi. Untuk kesederhanaan, dalam kasus kami, kami akan mengasumsikan bahwa blok terdiri dari sejumlah transaksi tetap, yaitu tiga dalam kasus ini. Karena blok perlu menyimpan daftar ketiga transaksi ini, kami akan mendeklarasikan variabel instan yang disebutVerified_transactions sebagai berikut :

```
self.verified_transactions = []
```
menamai variabel ini sebagai Verified_transactions untuk menunjukkan bahwa hanya transaksi valid yang diverifikasi yang akan ditambahkan ke blok. Setiap blok juga menyimpan nilai hash dari blok sebelumnya, sehingga rantai blok menjadi tidak berubah.



Untuk menyimpan hash sebelumnya, kami mendeklarasikan variabel instan sebagai berikut :

```
self.previous_block_hash = ""
```

mendeklarasikan satu variabel lagi yang disebut Nonce untuk menyimpan nonce yang dibuat oleh penambang selama proses penambangan.

```
self.Nonce = ""
```

In [ ]:
class Block:
   def __init__(self):
      self.verified_transactions = []
      self.previous_block_hash = ""
      self.Nonce = ""

Karena setiap blok membutuhkan nilai hash blok sebelumnya, kami mendeklarasikan variabel global yang disebut last_block_hash sebagai berikut :

```
last_block_hash = ""
```

<br>
<br>

<h2>6. Creating Genesis Block</h2>

Kami berasumsi bahwa pencetus TPCoin awalnya memberikan 500 TPCoin kepada klien yang dikenal Dinesh. Untuk ini, dia pertama kali membuat instance Dinesh

In [ ]:
Dinesh = Client()

Kami kemudian membuat transaksi genesis dan mengirim 500 TPCoin ke alamat publik Dinesh

In [ ]:
t0 = Transaction (
   "Genesis",
   Dinesh.identity,
   500.0
)

Sekarang membuat instance dari kelas Block dan menyebutnya block0.

In [ ]:
block0 = Block()

menginisialisasi variabel instance Previous_block_hash dan Nonce ke None, karena ini adalah transaksi pertama yang disimpan di blockchain.

block0.previous_block_hash = None
Nonce = None

Selanjutnya, kami akan menambahkan t0 transaksi di atas ke daftar Verified_transactions yang disimpan di dalam blok

In [ ]:
block0.verified_transactions.append (t0)

Pada titik ini, blok sepenuhnya diinisialisasi dan siap ditambahkan ke blockchain kami. Kami akan membuat blockchain untuk tujuan ini. Sebelum kita menambahkan blok ke blockchain, kita akan meng-hash blok tersebut dan menyimpan nilainya dalam variabel global yang disebut last_block_hash yang telah kita deklarasikan sebelumnya. Nilai ini akan digunakan oleh penambang berikutnya di bloknya.



Kami menggunakan dua baris pengkodean berikut untuk hashing blok dan menyimpan digest value

In [ ]:
digest = hash (block0)
last_block_hash = digest


<br>
<br>

<h2>7. Creating Blockchain</h2>

Blockchain berisi daftar blok yang dirantai satu sama lain. Untuk menyimpan seluruh daftar, kami akan membuat variabel daftar yang disebut TPCoins

In [ ]:
TPCoins = []

ini akan menulis metode utilitas yang disebut dump_blockchain untuk membuang konten seluruh blockchain. Kami pertama-tama mencetak panjang blockchain sehingga kami tahu berapa banyak blok yang saat ini ada di blockchain.

```
def dump_blockchain (self):
   print ("Number of blocks in the chain: " + str(len (self)))
```

<b>NOTE</b> : bahwa seiring berjalannya waktu, jumlah blok di blockchain akan sangat tinggi untuk dicetak. Jadi, ketika Anda mencetak konten blockchain, Anda mungkin harus memutuskan kisaran yang ingin Anda periksa. Dalam kode di bawah ini, kami telah mencetak seluruh blockchain karena kami tidak akan menambahkan terlalu banyak blok dalam demo saat ini.



Untuk beralih melalui rantai, kami menyiapkan loop for sebagai berikut:

```
for x in range (len(TPCoins)):
   block_temp = TPCoins[x] 
```

Setiap blok yang direferensikan disalin ke variabel sementara yang disebut block_temp.

Kami mencetak nomor blok sebagai judul untuk setiap blok. Perhatikan bahwa angka akan dimulai dengan nol, blok pertama adalah blok genesis yang diberi nomor nol.

```
print ("block # " + str(x))
```

Di dalam setiap blok, kami telah menyimpan daftar tiga transaksi (kecuali blok genesis) dalam variabel yang disebutVerified_transactions. Kami mengulangi daftar ini dalam for loop dan untuk setiap item yang diambil, kami memanggil fungsi display_transaction untuk menampilkan detail transaksi.

```
for transaction in block_temp.verified_transactions:
   display_transaction (transaction)
```

In [ ]:
def dump_blockchain (self):
   print ("Number of blocks in the chain: " + str(len (self)))
   for x in range (len(TPCoins)):
      block_temp = TPCoins[x]
      print ("block # " + str(x))
      for transaction in block_temp.verified_transactions:
         display_transaction (transaction)
         print ('--------------')
      print ('=====================================')

<b>NOTE</b> : bahwa di sini kita telah menyisipkan pemisah pada titik yang sesuai dalam kode untuk membatasi blok dan transaksi di dalamnya.


<br>
<br>

<h2>8. Adding Genesis Block</h2>

Menambahkan blok ke blockchain melibatkan penambahan blok yang dibuat ke daftar TPCoin kami

In [ ]:
TPCoins.append(block0)

<b>NOTE</b> : bahwa tidak seperti blok lainnya dalam sistem, blok genesis hanya berisi satu transaksi yang dimulai oleh pencetus sistem TPCoins. Sekarang, Anda akan membuang konten blockchain dengan memanggil fungsi global kami dump_blockchain

In [ ]:
dump_blockchain(TPCoins)

output :

```
Number of blocks in the chain: 1
block # 0
sender: Genesis
-----
recipient:
30819f300d06092a864886f70d010101050003818d0030818902818100ed272b52ccb539
e2cd779c6cc10ed1dfadf5d97c6ab6de90ed0372b2655626fb79f62d0e01081c163b0864
cc68d426bbe9438e8566303bb77414d4bfcaa3468ab7febac099294de10273a816f7047d
4087b4bafa11f141544d48e2f10b842cab91faf33153900c7bf6c08c9e47a7df8aa7e60d
c9e0798fb2ba3484bbdad2e4430203010001
-----
value: 500.0
-----
time: 2019-01-14 16:18:02.042739
-----
--------------
=====================================
```

Pada titik ini sistem blockchain siap digunakan. Kami sekarang akan memungkinkan klien yang tertarik untuk menjadi penambang dengan memberi mereka fungsi penambangan


<br>
<br>

<h2>9. Creating Miners</h2>

Untuk mengaktifkan penambangan, kita perlu mengembangkan fungsi penambangan. Fungsionalitas penambangan perlu menghasilkan digest pada string pesan yang diberikan dan memberikan bukti kerja

<h3>Message Digest Function</h3>

menulis fungsi utilitas yang disebut sha256 untuk membuat digest pada pesan yang diberikan

In [ ]:
def sha256(message):
    return hashlib.sha256(message.encode('ascii')).hexdigest()

Fungsi sha256 mengambil pesan sebagai parameter, mengkodekannya ke ASCII, menghasilkan intisari heksadesimal dan mengembalikan nilai ke pemanggil.

<h3>Mining Function</h3>

sekarang mengembangkan fungsi tambang yang menerapkan strategi penambangan sendiri. Strategi kami dalam hal ini adalah menghasilkan hash pada pesan yang diberikan yang diawali dengan angka 1 yang diberikan. Jumlah 1 yang diberikan ditentukan sebagai parameter untuk fungsi tambang yang ditentukan sebagai tingkat kesulitan.



Misalnya, jika Anda menentukan tingkat kesulitan 2, hash yang dihasilkan pada pesan tertentu harus dimulai dengan dua angka 1 - seperti 11xxxxxxxx. Jika tingkat kesulitannya adalah 3, hash yang dihasilkan harus dimulai dengan tiga angka 1 - seperti 111xxxxxxxxx. Mengingat persyaratan ini, kami sekarang akan mengembangkan fungsi penambangan seperti yang ditunjukkan pada langkah-langkah yang diberikan di bawah ini.

- Langkah 1

  Fungsi penambangan mengambil dua parameter - pesan dan tingkat kesulitan..
  ```
  def mine(message, difficulty=1):
  ```
  
- Langkah 2
  
  Tingkat kesulitan harus lebih besar atau sama dengan 1, kami memastikan ini dengan pernyataan tegas berikut:
  ```
  assert difficulty >= 1
  ```
  
- Langkah 3
  
  Kami membuat variabel awalan menggunakan tingkat kesulitan yang ditetapkan.
  ```
  prefix = '1' * difficulty
  ```
  
<b>NOTE</b> : jika tingkat kesulitannya adalah 2, awalannya adalah "11" dan jika tingkat kesulitannya adalah 3, awalannya adalah "111", dan seterusnya. Kami akan memeriksa apakah awalan ini ada di intisari pesan yang dihasilkan. Untuk mencerna pesan itu sendiri, kami menggunakan dua baris kode berikut:

```
for i in range(1000):
   digest = sha256(str(hash(message)) + str(i))
```

menambahkan nomor baru i ke hash pesan di setiap iterasi dan menghasilkan digest baru pada pesan gabungan. Saat input ke fungsi sha256 berubah di setiap iterasi, nilai digest juga akan berubah. Kami memeriksa apakah nilai digest ini memiliki awalan yang ditetapkan di atas.

```
if digest.startswith(prefix):
```

  

In [ ]:
def mine(message, difficulty=1):
   assert difficulty >= 1
   prefix = '1' * difficulty
   for i in range(1000):
      digest = sha256(str(hash(message)) + str(i))
      if digest.startswith(prefix):
         print ("after " + str(i) + " iterations found nonce: "+ digest)
      return digest

ini telah menambahkan pernyataan print yang mencetak nilai digest dan jumlah iterasi yang diperlukan untuk memenuhi kondisi sebelum kembali dari fungsi.


<h3>Testing Mining Function</h3>

Untuk menguji fungsi penambangan kami, cukup jalankan pernyataan berikut :

In [ ]:
mine("test message", 2)

Ketika Anda menjalankan kode di atas, Anda akan melihat output yang mirip dengan yang di bawah ini :

```
after 138 iterations found nonce:
11008a740eb2fa6bf8d55baecda42a41993ca65ce66b2d3889477e6bfad1484c
```

<b>NOTE</b> : bahwa digest yang dihasilkan dimulai dengan "11". Jika Anda mengubah tingkat kesulitan menjadi 3, digest yang dihasilkan akan dimulai dengan “111”, dan tentu saja, mungkin akan memerlukan lebih banyak iterasi. Seperti yang Anda lihat, penambang dengan kekuatan pemrosesan lebih akan dapat menambang pesan yang diberikan sebelumnya. Begitulah cara para penambang bersaing satu sama lain untuk mendapatkan pendapatan mereka.


<br>
<br>

<h2>10. Adding Blocks</h2>

Setiap penambang akan mengambil transaksi dari kumpulan transaksi yang dibuat sebelumnya. Untuk melacak jumlah pesan yang sudah ditambang, kita harus membuat variabel global

In [ ]:
last_transaction_index = 0

sekarang akan memiliki penambang pertama kami menambahkan blok ke blockchain

<h3>Adding First Block</h3>

Untuk menambahkan blok baru, pertama-tama kita membuat instance dari kelas Block.

```
block = Block()
```
ambil 3 transaksi teratas dari antrian

```
for i in range(3):
   temp_transaction = transactions[last_transaction_index]
   # validate transaction
```

Sebelum menambahkan transaksi ke blok, penambang akan memverifikasi validitas transaksi. Validitas transaksi diverifikasi dengan menguji kesetaraan hash yang diberikan oleh pengirim terhadap hash yang dihasilkan oleh penambang menggunakan kunci publik pengirim. Selain itu, penambang akan memverifikasi bahwa pengirim memiliki saldo yang cukup untuk membayar transaksi saat ini.

Untuk singkatnya, kami belum menyertakan fungsi ini dalam tutorial. Setelah transaksi divalidasi, kami menambahkannya ke daftarVerified_transactions di instance blok.

```
block.verified_transactions.append (temp_transaction)
```

naikkan indeks transaksi terakhir sehingga penambang berikutnya akan mengambil transaksi berikutnya dalam antrian.

```
last_transaction_index += 1
```

tambahkan tepat tiga transaksi ke blok. Setelah ini selesai, kami akan menginisialisasi sisa variabel instan dari kelas Block. Kami pertama-tama menambahkan hash dari block terakhir.

```
block.previous_block_hash = last_block_hash
```

Selanjutnya kita menambang blok tersebut dengan tingkat kesulitan 2

```
block.Nonce = mine (block, 2)
```

<b>NOTE</b> : bahwa parameter pertama untuk fungsi tambang adalah objek biner. Kami sekarang hash seluruh blok dan membuat intisari di atasnya.

```
digest = hash (block)
```

Akhirnya, kami menambahkan blok yang dibuat ke blockchain dan menginisialisasi ulang variabel global last_block_hash untuk digunakan di block berikutnya

In [ ]:
block = Block()
for i in range(3):
   temp_transaction = transactions[last_transaction_index]
   # validate transaction
   # if valid
   block.verified_transactions.append (temp_transaction)
   last_transaction_index += 1

block.previous_block_hash = last_block_hash
block.Nonce = mine (block, 2)
digest = hash (block)
TPCoins.append (block)
last_block_hash = digest


<h3>Adding More Blocks</h3>

sekarang akan menambahkan dua blok lagi ke blockchain kami. Kode untuk menambahkan dua blok berikutnya diberikan di bawah ini

In [ ]:
# Miner 2 adds a block
block = Block()

for i in range(3):
   temp_transaction = transactions[last_transaction_index]
   # validate transaction
   # if valid
   block.verified_transactions.append (temp_transaction)
   last_transaction_index += 1
block.previous_block_hash = last_block_hash
block.Nonce = mine (block, 2)digest = hash (block)
TPCoins.append (block)last_block_hash = digest
# Miner 3 adds a block
block = Block()

for i in range(3):
   temp_transaction = transactions[last_transaction_index]
   #display_transaction (temp_transaction)
   # validate transaction
   # if valid
   block.verified_transactions.append (temp_transaction)
   last_transaction_index += 1

block.previous_block_hash = last_block_hash
block.Nonce = mine (block, 2)
digest = hash (block)

TPCoins.append (block)
last_block_hash = digest

Saat Anda menambahkan dua blok ini, Anda juga akan melihat jumlah iterasi yang diperlukan untuk menemukan Nonce. Pada titik ini, blockchain kami terdiri dari 4 blok termasuk blok genesis.

<h3>Dumping Entire Blockchain</h3>

memverifikasi isi seluruh blockchain menggunakan pernyataan berikut :


In [ ]:
dump_blockchain(TPCoins)

output :

```
Number of blocks in the chain: 4
block # 0
sender: Genesis
-----
recipient:
30819f300d06092a864886f70d010101050003818d0030818902818100ed272b52ccb539e2cd779
c6cc10ed1dfadf5d97c6ab6de90ed0372b2655626fb79f62d0e01081c163b0864cc68d426bbe943
8e8566303bb77414d4bfcaa3468ab7febac099294de10273a816f7047d4087b4bafa11f141544d4
8e2f10b842cab91faf33153900c7bf6c08c9e47a7df8aa7e60dc9e0798fb2ba3484bbdad2e44302
03010001
-----
value: 500.0
-----
time: 2019-01-14 16:18:02.042739
-----
--------------
=====================================
block # 1
sender:
30819f300d06092a864886f70d010101050003818d0030818902818100bb064c99c492144a9f463
480273aba93ac1db1f0da3cb9f3c1f9d058cf499fd8e54d244da0a8dd6ddd329ec86794b04d773e
b4841c9f935ea4d9ccc2821c7a1082d23b6c928d59863407f52fa05d8b47e5157f8fe56c2ce3279
c657f9c6a80500073b0be8093f748aef667c03e64f04f84d311c4d866c12d79d3fc3034563dfb02
03010001
-----
recipient:
30819f300d06092a864886f70d010101050003818d0030818902818100be93b516b28c6e674abe7
abdb11ce0fdf5bb728b75216b73f37a6432e4b402b3ad8139b8c0ba541a72c8add126b6e1a1308f
b98b727beb63c6060356bb177bb7d54b54dbe87aee7353d0a6baa9397704de625d1836d3f42c7ee
5683f6703259592cc24b09699376807f28fe0e00ff882974484d805f874260dfc2d1627473b9102
03010001
-----
value: 15.0
-----
time: 2019-01-14 16:18:01.859915
-----
--------------
sender:
30819f300d06092a864886f70d010101050003818d0030818902818100bb064c99c492144a9f463
480273aba93ac1db1f0da3cb9f3c1f9d058cf499fd8e54d244da0a8dd6ddd329ec86794b04d773e
b4841c9f935ea4d9ccc2821c7a1082d23b6c928d59863407f52fa05d8b47e5157f8fe56c2ce3279
c657f9c6a80500073b0be8093f748aef667c03e64f04f84d311c4d866c12d79d3fc3034563dfb02
03010001
-----
recipient:
30819f300d06092a864886f70d010101050003818d0030818902818100a070c82b34ae143cbe59b
3a2afde7186e9d5bc274955d8112d87a00256a35369acc4d0edfe65e8f9dc93fbd9ee74b9e7ea12
334da38c8c9900e6ced1c4ce93f86e06611e656521a1eab561892b7db0961b4f212d1fd5b5e49ae
09cf8c603a068f9b723aa8a651032ff6f24e5de00387e4d062375799742a359b8f22c5362e56502
03010001
-----
value: 6.0
-----
time: 2019-01-14 16:18:01.860966
-----
--------------
sender:
30819f300d06092a864886f70d010101050003818d0030818902818100be93b516b28c6e674abe7
abdb11ce0fdf5bb728b75216b73f37a6432e4b402b3ad8139b8c0ba541a72c8add126b6e1a1308f
b98b727beb63c6060356bb177bb7d54b54dbe87aee7353d0a6baa9397704de625d1836d3f42c7ee
5683f6703259592cc24b09699376807f28fe0e00ff882974484d805f874260dfc2d1627473b9102
03010001
-----
recipient:
30819f300d06092a864886f70d010101050003818d0030818902818100cba097c0854876f41338c
62598c658f545182cfa4acebce147aedf328181f9c4930f14498fd03c0af6b0cce25be99452a81d
f4fa30a53eddbb7bb7b203adf8764a0ccd9db6913a576d68d642d8fd47452590137869c25d9ff83
d68ebe6d616056a8425b85b52e69715b8b85ae807b84638d8f00e321b65e4c33acaf6469e18e302
03010001
-----
value: 2.0
-----
time: 2019-01-14 16:18:01.861958
-----
--------------
=====================================
block # 2
sender:
30819f300d06092a864886f70d010101050003818d0030818902818100a070c82b34ae143cbe59b
3a2afde7186e9d5bc274955d8112d87a00256a35369acc4d0edfe65e8f9dc93fbd9ee74b9e7ea12
334da38c8c9900e6ced1c4ce93f86e06611e656521a1eab561892b7db0961b4f212d1fd5b5e49ae
09cf8c603a068f9b723aa8a651032ff6f24e5de00387e4d062375799742a359b8f22c5362e56502
03010001
-----
recipient:
30819f300d06092a864886f70d010101050003818d0030818902818100be93b516b28c6e674abe7
abdb11ce0fdf5bb728b75216b73f37a6432e4b402b3ad8139b8c0ba541a72c8add126b6e1a1308f
b98b727beb63c6060356bb177bb7d54b54dbe87aee7353d0a6baa9397704de625d1836d3f42c7ee
5683f6703259592cc24b09699376807f28fe0e00ff882974484d805f874260dfc2d1627473b9102
03010001
-----
value: 4.0
-----
time: 2019-01-14 16:18:01.862946
-----
--------------
sender:
30819f300d06092a864886f70d010101050003818d0030818902818100cba097c0854876f41338c
62598c658f545182cfa4acebce147aedf328181f9c4930f14498fd03c0af6b0cce25be99452a81d
f4fa30a53eddbb7bb7b203adf8764a0ccd9db6913a576d68d642d8fd47452590137869c25d9ff83
d68ebe6d616056a8425b85b52e69715b8b85ae807b84638d8f00e321b65e4c33acaf6469e18e302
03010001
-----
recipient:
30819f300d06092a864886f70d010101050003818d0030818902818100a070c82b34ae143cbe59b
3a2afde7186e9d5bc274955d8112d87a00256a35369acc4d0edfe65e8f9dc93fbd9ee74b9e7ea12
334da38c8c9900e6ced1c4ce93f86e06611e656521a1eab561892b7db0961b4f212d1fd5b5e49ae
09cf8c603a068f9b723aa8a651032ff6f24e5de00387e4d062375799742a359b8f22c5362e56502
03010001
-----
value: 7.0
-----
time: 2019-01-14 16:18:01.863932
-----
--------------
sender:
30819f300d06092a864886f70d010101050003818d0030818902818100be93b516b28c6e674abe7
abdb11ce0fdf5bb728b75216b73f37a6432e4b402b3ad8139b8c0ba541a72c8add126b6e1a1308f
b98b727beb63c6060356bb177bb7d54b54dbe87aee7353d0a6baa9397704de625d1836d3f42c7ee
5683f6703259592cc24b09699376807f28fe0e00ff882974484d805f874260dfc2d1627473b9102
03010001
-----
recipient:
30819f300d06092a864886f70d010101050003818d0030818902818100a070c82b34ae143cbe59b
3a2afde7186e9d5bc274955d8112d87a00256a35369acc4d0edfe65e8f9dc93fbd9ee74b9e7ea12
334da38c8c9900e6ced1c4ce93f86e06611e656521a1eab561892b7db0961b4f212d1fd5b5e49ae
09cf8c603a068f9b723aa8a651032ff6f24e5de00387e4d062375799742a359b8f22c5362e56502
03010001
-----
value: 3.0
-----
time: 2019-01-14 16:18:01.865099
-----
--------------
=====================================
block # 3
sender:
30819f300d06092a864886f70d010101050003818d0030818902818100a070c82b34ae143cbe59b
3a2afde7186e9d5bc274955d8112d87a00256a35369acc4d0edfe65e8f9dc93fbd9ee74b9e7ea12
334da38c8c9900e6ced1c4ce93f86e06611e656521a1eab561892b7db0961b4f212d1fd5b5e49ae
09cf8c603a068f9b723aa8a651032ff6f24e5de00387e4d062375799742a359b8f22c5362e56502
03010001
-----
recipient:
30819f300d06092a864886f70d010101050003818d0030818902818100bb064c99c492144a9f463
480273aba93ac1db1f0da3cb9f3c1f9d058cf499fd8e54d244da0a8dd6ddd329ec86794b04d773e
b4841c9f935ea4d9ccc2821c7a1082d23b6c928d59863407f52fa05d8b47e5157f8fe56c2ce3279
c657f9c6a80500073b0be8093f748aef667c03e64f04f84d311c4d866c12d79d3fc3034563dfb02
03010001
-----
value: 8.0
-----
time: 2019-01-14 16:18:01.866219
-----
--------------
sender:
30819f300d06092a864886f70d010101050003818d0030818902818100a070c82b34ae143cbe59b
3a2afde7186e9d5bc274955d8112d87a00256a35369acc4d0edfe65e8f9dc93fbd9ee74b9e7ea12
334da38c8c9900e6ced1c4ce93f86e06611e656521a1eab561892b7db0961b4f212d1fd5b5e49ae
09cf8c603a068f9b723aa8a651032ff6f24e5de00387e4d062375799742a359b8f22c5362e56502
03010001
-----
recipient:
30819f300d06092a864886f70d010101050003818d0030818902818100be93b516b28c6e674abe7
abdb11ce0fdf5bb728b75216b73f37a6432e4b402b3ad8139b8c0ba541a72c8add126b6e1a1308f
b98b727beb63c6060356bb177bb7d54b54dbe87aee7353d0a6baa9397704de625d1836d3f42c7ee
5683f6703259592cc24b09699376807f28fe0e00ff882974484d805f874260dfc2d1627473b9102
03010001
-----
value: 1.0
-----
time: 2019-01-14 16:18:01.867223
-----
--------------
sender:
30819f300d06092a864886f70d010101050003818d0030818902818100cba097c0854876f41338c
62598c658f545182cfa4acebce147aedf328181f9c4930f14498fd03c0af6b0cce25be99452a81d
f4fa30a53eddbb7bb7b203adf8764a0ccd9db6913a576d68d642d8fd47452590137869c25d9ff83
d68ebe6d616056a8425b85b52e69715b8b85ae807b84638d8f00e321b65e4c33acaf6469e18e302
03010001
-----
recipient: 
30819f300d06092a864886f70d010101050003818d0030818902818100bb064c99c492144a9f463
480273aba93ac1db1f0da3cb9f3c1f9d058cf499fd8e54d244da0a8dd6ddd329ec86794b04d773e
b4841c9f935ea4d9ccc2821c7a1082d23b6c928d59863407f52fa05d8b47e5157f8fe56c2ce3279
c657f9c6a80500073b0be8093f748aef667c03e64f04f84d311c4d866c12d79d3fc3034563dfb02
03010001
-----
value: 5.0
-----
time: 2019-01-14 16:18:01.868241
-----
--------------
=====================================
```



<br>
<br>

<h2>11. Scope</h2>

kita telah belajar bagaimana membangun sebuah proyek blockchain dengan Python. Ada banyak area di mana Anda perlu menambahkan fungsionalitas lebih lanjut ke proyek ini.

Misalnya, perlu menulis fungsi untuk mengelola antrian transaksi. Setelah transaksi ditambang dan blok yang ditambang diterima oleh sistem, mereka tidak perlu disimpan lagi.

Selain itu, para penambang tentu akan lebih memilih untuk mengambil transaksi dengan biaya tertinggi. Pada saat yang sama, Anda harus memastikan bahwa transaksi dengan biaya rendah atau tanpa biaya tidak akan kelaparan selamanya.

Anda perlu mengembangkan algoritma untuk mengelola antrian. ini tidak menyertakan kode antarmuka klien. Anda perlu mengembangkan ini untuk klien normal dan penambang. Proyek blockchain yang lengkap akan mengalami beberapa baris kode lagi dan berada di luar cakupan ini.
